Optimización usando LASSO
===

En esta técnica, el término de penalización es el valor absoluto de los coeficientes del modelo de regresión.

$$ \sum_{i=1}^N (y_i -  g(x_i))^2 + \alpha \sum_{p=1}^P ||w_p|| $$


$\alpha$ es un hiperparámetro suministrado por el usuario. **Nota:** Para el modelo lineal, la penalización solo aplica para los coeficientes de $x$, no para el intercepto.

In [1]:
#
# A continuación se presenta la implementación de un modelo de
# regresión lineal que usa la función de penalización LASSO para
# estimar los parámetros óptimos. Complete el código presentado
# para que pasen las pruebas definidas en las celdas restantes.
#
import numpy as np
import pandas as pd
import pytest


class LassoRegression:
    def __init__(self, intercept, coef, maxiter, mu, alpha):
        self.intercept_ = intercept
        self.coef_ = np.array(coef)
        self._maxiter = maxiter
        self._mu = mu
        self._alpha = alpha
        self._grad_coef = np.array(coef)
        self._grad_intercept = intercept

    def compute_loss(self, x, y):
        d = self.g(x)
        return  np.sum([(yi - di)**2 for yi, di in zip(y, d)]) + self._alpha * np.sum(self.coef_)

    def g(self, x):
        return [self.coef_[0] * d[0] + self.coef_[1] * d[1] + self.intercept_ for d in x]

    def predict(self, x):
        return self.g(x)

    def compute_gradient(self, x, y):
        d = [self.coef_[0] * d[0] + self.coef_[1] * d[1] + self.intercept_ for d in x]
        e = [yi - di for yi, di in zip(y, d)]

        self._grad_coef[0] = -2 * np.sum([ei * xi[0] for xi, ei in zip(x, e)]) + self._alpha * np.where(self.coef_[0] > 0, 1, -1)
        self._grad_coef[1] = -2 * np.sum([ei * xi[1] for xi, ei in zip(x, e)]) + self._alpha * np.where(self.coef_[1] > 0, 1, -1)

        self._grad_intercept = - 2 * np.sum(e) 

    def fit(self, x, y):
        for iter in range(self._maxiter):
            self.compute_gradient(x, y)
            self.improve()

    def improve(self):
        self.intercept_ = self.intercept_ - self._mu * self._grad_intercept
        self.coef_ = self.coef_ - self._mu * self._grad_coef


x = [
    [0.0, 0.1],
    [0.2, 0.3],
    [0.4, 0.5],
    [0.6, 0.7],
    [0.8, 0.9],
    [1.0, 1.1],
]

# y = 1 x1 + 1.1 x2 + 0.2

# (y - w0 x1 + w1 x2 + b)^2 + a * (|w0|+|w1|)

y = [
    0.31,
    0.73,
    1.15,
    1.57,
    1.99,
    2.41,
]

In [2]:
#
# Test 1
# =============================================================================
# Implemente la función de pérdida.
#
# Rta/
# True
#

# ---->>> Evaluación ---->>>
lr = LassoRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    alpha=100,
)

pytest.approx(lr.compute_loss(x, y), 0.0001) == 57.5544

True

In [3]:
#
# Test 2
# =============================================================================
# Implemente la función de pronóstico
#
# Rta/
# True
#

# ---->>> Evaluación ---->>>
lr = LassoRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    alpha=100,
)

all(
    pytest.approx(a) == b
    for a, b in zip(lr.predict(x), [0.13, 0.23, 0.33, 0.43, 0.53, 0.63])
)

True

In [4]:
#
# Test 3
# =============================================================================
# Implemente el gradiente
#
# Rta/
# True
# True
#

# ---->>> Evaluación ---->>>
lr = LassoRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    alpha=100,
)
lr.compute_gradient(x, y)

print(lr._grad_intercept == pytest.approx(-11.76))
print(all(pytest.approx(a) == b for a, b in zip(lr._grad_coef, [91.88 , 90.704])))

True
True


In [5]:
#
# Test 4
# =============================================================================
# Implemente la función fit
#
# Rta/
# True
# True
#

# ---->>> Evaluación ---->>>
lr = LassoRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=1000,
    mu=0.001,
    alpha=100,
)
lr.fit(x, y)
print(pytest.approx(lr.intercept_, 0.001) == 1.356084)
print(all(pytest.approx(a, 0.001) == b for a, b in zip(lr.coef_, [-0.045481 , -0.019872])))

True
True
